## CSCI S-89 Introduction to Deep Learning
Harvard Summer School 2025

# Final Project:
# *Face Recognition with Deep Learning*

The goal of this project is to detect human faces in images and recognize who the faces belong to from a predefined group of people.

# Part III: **Prepare and Preprocess Input Data for the Model**

Next, we will turn the input face images into the format that the model expects to start training.

We will create 2 new folders, one for training and one for validation, and we will split our input data into them.<br>
We will use 80% of the data for training and 20% for validation.

In [ ]:
import os

# Get the original path to the images.
original_dataset = os.path.join(project_path, 'face_recognition_dataset', 'peoples_faces', 'peoples_faces')

# Path where we will put the training and the validation folders.
target_folder = os.path.join(project_path, 'face_recognition_dataset', 'peoples_faces')

# Create the training folder.
train_dir = os.path.join(target_folder, 'train')
os.mkdir(train_dir)

# Create the validation folder.
val_dir = os.path.join(target_folder, 'validation')
os.mkdir(val_dir)

In [ ]:
import random
import shutil

# For every person in the original dataset...
for person in os.listdir(original_dataset):
  # Take the person's folder.
  persons_folder = os.path.join(original_dataset, person)

  # Shuffle the images in the folder
  # to make everything fairer.
  images = os.listdir(persons_folder)
  random.shuffle(images)

  # Split the images into training and validation subsets.
  split_point = int(len(images) * 0.8)
  train_images = images[:split_point]
  validation_images = images[split_point:]

  # Create new folders for the person
  # in the training and the validation folders.
  train_person_dir = os.path.join(train_dir, person)
  os.mkdir(train_person_dir)

  val_person_dir = os.path.join(val_dir, person)
  os.mkdir(val_person_dir)

  # Copy training images to train folder.
  for img in train_images:
    current_image_path = os.path.join(persons_folder, img)
    target_path = os.path.join(train_person_dir, img)
    shutil.copy(current_image_path, target_path)

  # Copy validation images to validation folder.
  for img in validation_images:
    current_image_path = os.path.join(persons_folder, img)
    target_path = os.path.join(val_person_dir, img)
    shutil.copy(current_image_path, target_path)

For the following preprocessing steps I used the online tensorflow documentation (https://www.tensorflow.org/tutorials/images/classification).

We will format the input data accordingly for the model:
1.   We will define that every person in the dataset is a class.
2.   We will give labels to the images (the name of the person, a.k.a. the name of the folder).
3.   We will load the images in batches (32).
4.   We will make sure all images are 224x224 pixels.

In [ ]:
import os

target_folder = os.path.join(project_path, 'face_recognition_dataset', 'peoples_faces')
train_dir = os.path.join(target_folder, 'train')
val_dir = os.path.join(target_folder, 'validation')

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Format images for training.
train_ds = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical', # one-hot encoded labels for Categorical Crossentropy
    shuffle=True
)

# Format images for validation.
val_ds = image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    shuffle=True
)

Found 1280 files belonging to 20 classes.
Found 320 files belonging to 20 classes.


In [ ]:
for images, labels in train_ds:
  print('Train Dataset:')
  print(f'  Image Shape: {images.shape}')
  print(f'  Label Shape: {labels.shape}')
  print()
  break

for images, labels in val_ds:
  print('Validation Dataset:')
  print(f'  Image Shape: {images.shape}')
  print(f'  Label Shape: {labels.shape}')
  break

Train Dataset:
  Image Shape: (32, 224, 224, 3)
  Label Shape: (32, 20)

Validation Dataset:
  Image Shape: (32, 224, 224, 3)
  Label Shape: (32, 20)


And of course we will **scale the data**.

In [ ]:
from tensorflow.keras import layers

normalization_layer = layers.Rescaling(1./255)

# Perform pixel normalization.
train_ds_scaled = train_ds.map(lambda x, y: (normalization_layer(x), y))

val_ds_scaled = val_ds.map(lambda x, y: (normalization_layer(x), y))

And we do **data augmentation** to prevent overfitting and improve generalization (only to the training data).

In [ ]:
from tensorflow.keras import layers, Sequential

data_augmentation = Sequential([
    layers.RandomRotation(0.1),
    layers.RandomTranslation(0.2, 0.2),
    layers.RandomZoom(0.2),
    layers.RandomFlip('horizontal'),
    layers.RandomContrast(0.2),
])

train_ds_scaled_aug = train_ds_scaled.map(lambda x, y: (data_augmentation(x), y))

And now we are ready to build, compile, and train our face recognition model.